# Damaging User Ids

# Initialization

In [ ]:
!pip3 install xlrd
!pip3 install openpyxl

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud_fa import WordCloudFa
from matplotlib.pyplot import figure

import re
import copy
import time
import math
import datetime
import hazm
from hazm import stopwords_list
from hazm import word_tokenize
from cleantext import clean
from nltk.corpus import stopwords
from collections import Counter

import itertools

import scipy.stats as stats
from tqdm.notebook import tqdm

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
tqdm.pandas()

In [ ]:
base_path = "./data/"

damage_path = 'damages.xlsx'
all_damages_path = 'all_damages.xlsx'
insurance_path = 'expert_insurance_2100.xlsx'
all_achare_path = 'Achar Database.xlsx'

# Data reading

In [ ]:
damage_df = pd.read_excel(base_path + damage_path, engine='openpyxl', dtype=str)
damage_df.rename(columns={'شماره متخصص': 'phone_number', 'متخصص': 'expert_name'}, inplace=True)
damage_df

In [ ]:
all_damages_df = pd.read_excel(base_path + all_damages_path, engine='openpyxl', dtype=str)
all_damages_df.rename(columns={'شماره متخصص': 'phone_number', 'اسم متخصص': 'expert_name', 'کد پیگیری': 'tracking-code'}, inplace=True)
all_damages_df['phone_number'] = all_damages_df['phone_number'].progress_apply(lambda d: '0' + d[2:])
all_damages_df

In [ ]:
insurance_df = pd.read_excel(base_path + insurance_path, engine='openpyxl', dtype=str)
insurance_df.rename(columns={"شماره تلفن همراه',": 'phone_number', 'نام': 'expert_fname', 'نام خانوادگی': 'expert_lname', 'کد ملی': 'national_code', 'شماره شناسنامه': 'national_number'}, inplace=True)
insurance_df['phone_number'] = insurance_df['phone_number'].progress_apply(lambda d: '0' + d[2:])
insurance_df

In [ ]:
achare_df = pd.read_excel(base_path + all_achare_path, engine='openpyxl', usecols=['نام و نام خانوادگی', 'کد ملی', 'شماره تلفن ثابت', 'شماره تلفن همراه', 'محل تولد', 'تاریخ تولد', 'شماره شناسنامه', 'تاریخ عضویت', 'نوع همکاری', 'زمینه کاری اصلی 1', 'زمینه کاری فرعی 1'], dtype = str) 
achare_df.rename(columns={"شماره تلفن همراه": 'phone_number', 'نام و نام خانوادگی': 'expert_name', 'کد ملی': 'national_code', 'شماره شناسنامه': 'national_number'}, inplace=True)
achare_df['phone_number'] = achare_df['phone_number'].progress_apply(lambda n: str(n))
achare_df['national_code'] = achare_df['national_code'].progress_apply(lambda i: str(i))
achare_df = achare_df[achare_df['national_code'] != 'nan']
achare_df

In [ ]:
damage_df['phone_number'] = damage_df['phone_number'].apply(lambda n: re.sub('\D', '', n))
all_damages_df['phone_number'] = all_damages_df['phone_number'].apply(lambda n: re.sub('\D', '', n))
insurance_df['phone_number'] = insurance_df['phone_number'].apply(lambda n: re.sub('\D', '', n))
achare_df['phone_number'] = achare_df['phone_number'].apply(lambda n: re.sub('\D', '', n))

# Required users

In [ ]:
damaging_users = pd.DataFrame({})

In [ ]:
damaging_users['expert_name'] = damage_df['expert_name'].append(all_damages_df['expert_name'])
damaging_users['phone_number'] = damage_df['phone_number'].append(all_damages_df['phone_number'])
damaging_users = damaging_users.drop_duplicates(subset=['phone_number'], keep='first')
damaging_users

In [ ]:
available_in_2100 = set(insurance_df['phone_number'].tolist())
available_in_achare = set(achare_df['phone_number'].tolist())

In [ ]:
available_users_in_2100 = damaging_users[damaging_users['phone_number'].isin(available_in_2100)]
not_available_users_in_2100 = damaging_users.drop(available_users_in_2100.index)
not_available_users_in_2100

In [ ]:
available_users_in_achare = not_available_users_in_2100[not_available_users_in_2100['phone_number'].isin(available_in_achare)]
not_available_users_in_achare = not_available_users_in_2100.drop(available_users_in_achare.index)

In [ ]:
not_available_users_in_achare.iloc[6]['phone_number']

In [ ]:
available_users_in_achare

In [ ]:
not_available_users_in_achare

In [ ]:
def get_achare_info(provided_info, col_name, provided_col='phone_number'):
    achare_info = achare_df[achare_df[provided_col] == provided_info].iloc[0]
    return achare_info[col_name]

In [ ]:
available_users_in_achare['national_code'] = available_users_in_achare['phone_number'].progress_apply(lambda n: get_achare_info(n, 'national_code'))
available_users_in_achare['national_number'] = available_users_in_achare['phone_number'].progress_apply(lambda n: get_achare_info(n, 'national_number'))
available_users_in_achare

In [ ]:
available_users_in_achare.to_csv(base_path + 'achare_available_users.csv', mode='w', encoding='utf-8', index=False)
not_available_users_in_achare.to_csv(base_path + 'achare_required_users.csv', mode='w', encoding='utf-8', index=False)